# Dense Retriver 실험

In [1]:
import torch
from torch.utils.data import DataLoader

from dense_retrieval import DenseRetrieval

In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from typing import List, Tuple, NoReturn, Any, Optional, Union

import torch
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from datasets import Dataset, load_from_disk, concatenate_datasets

from retrieval import SparseRetrieval, timer

## 1. 데이터 로드

In [3]:
import os
import json 

data_path  = "../data/"
dataset_path = "../data/train_dataset"
context_path = "wikipedia_documents.json"
model_checkpoint = "klue/bert-base"

org_dataset = load_from_disk(dataset_path)
full_ds = concatenate_datasets([
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ])

with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

Loading cached processed dataset at ../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [4]:
df_train = pd.DataFrame(org_dataset['train'])
#df_train = df_train[['document_id','title','answers','question','context', 'id','__index_level_0__']]
df_train = df_train[['answers']]
df_train.to_csv('./data/train_answers.csv')

df_valid = pd.DataFrame(org_dataset['validation'])
#df_valid = df_valid[['document_id','title','answers','question','context', 'id','__index_level_0__']]
df_valid = df_valid[['answers']]
df_valid.to_csv('./data/valid_answers.csv')

In [5]:
len(contexts)

56737

[s for s in contexts if org_dataset["validation"][0]['context'][0:5] in s]

## 2. 추론

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=False,)

args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

### 학습데이터 생성

In [7]:
dense_retriever = DenseRetrieval(tokenize_fn=tokenizer.tokenize, data_path = data_path, 
                                context_path = context_path, dataset_path=dataset_path, 
                                tokenizer=tokenizer, train_data=org_dataset["train"], is_bm25=True)

## 학습과정 ##
# train_dataset = dense_retriever.make_train_data(tokenizer)
# train_dataset = dense_retriever.load_train_data()
# dense_retriever.init_model(model_checkpoint)
# dense_retriever.train(args, train_dataset)

Lengths of unique wiki contexts : 56737
Embedding pickle load.


In [8]:
## 추론과정 ##
dense_retriever.load_model(model_checkpoint, "outputs/p_encoder_14.pt", "outputs/q_encoder_14.pt")
dense_retriever.get_dense_embedding()
# with open("./data/dense_embedding.bin", "rb") as f:
#     dense_retriever.dense_p_embedding = pickle.load(f)
#df = dense_retriever.retrieve(full_ds[0]['question'], topk=3)

cuda:0


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

load_model finished...


100%|██████████| 14184/14184 [14:16<00:00, 16.55it/s]


get_dense_embedding finished...


import pickle

with open("./data/dense_embedding.bin", "wb") as f:
    pickle.dump(dense_retriever.dense_p_embedding, f)

In [9]:
dense_retriever.dense_p_embedding.shape

torch.Size([56736, 768])

## 3. 실험 결과 확인

In [10]:
dense_retriever.get_relevant_doc(org_dataset["train"]["question"][0], k=1)

([215.40744018554688], tensor([14489]))

In [11]:
result_retriever = dense_retriever.retrieve(org_dataset["validation"], topk=5)
result_retriever

100%|██████████| 60/60 [00:04<00:00, 12.66it/s]


[query exhaustive search] done in 8.872 s


,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"[56287, 17338, 51123, 4634, 5078]","호르스트 제호퍼 주총리 밑에서 재무장관을 지냈으며, 주를 대변하는 연방 상원의원으로...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"[19569, 18413, 22775, 46742, 42504]",이 역이 교통 허브로 거듭나게 된 것은 거의 우연에 가까웠으며 기존에 스카버러 타운...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"[9111, 47398, 22525, 35100, 25391]","비밀결사란, 존재 자체가 구성원에 의해 은닉되거나 설사 공개되어도 그곳의 구성원이라...","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"[55652, 34724, 36142, 24446, 43210]",던스터블 백조 장신구는 1400년 무렵 제작된 백조 형태의 브로치이다. 영국이나 프...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"[4847, 29185, 42685, 56164, 50229]","집단 극화\n사람들은 개별적으로 토론할 때는 서로 타협을 볼 수 있지만, 양립하는 ...",탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"[4635, 34080, 4636, 22595, 5291]","고조선이 최초로 기록에 등장하는 시기는 기원전 7세기로, 이 무렵의 사실을 기록한 ...","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"[4615, 27210, 18616, 10770, 19239]",캐나다계 미국인 제임스 네이스미스가 고안하였다. 양팀 각 5 명씩의 선수가 한 개의...,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"[31255, 20252, 10707, 9235, 31256]",진행된 행성들의 주요한 의미는 그것들이 출생 천궁도의 각을 맺을 때 나타난다. 일반...,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"[52504, 22717, 51615, 45596, 42504]",빌카밤바가 위치한 라 컨벤시온 지방은 안데스산맥의 북동쪽에 있는 산줄기에 자리잡고 ...,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"


In [12]:
for i in range(10):
    df = dense_retriever.retrieve(org_dataset['validation'][i]['question'], topk=3)
    print(df)

[Search query]
 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? 

Top-1 passage with score 177.414993
호르스트 제호퍼 주총리 밑에서 재무장관을 지냈으며, 주를 대변하는 연방 상원의원으로서 상원 재무위원회 소속이었다.\n\n재무장관 시절 유럽 연합 집행위원회의 일괄 지원을 받고자 부실 주 지원 대출은행인 바이에른LB의 재건을 감독하기도 했다. 2014년에는 바이에른LB를 압박하여 헝가리 측에 MKB 단위를 매각함으로서 20여년 간 20억 유로의 손실을 초래한 부실투자를 종식시키기도 했다. 2015년에는 한스 외르크 셸링 오스트리아 외무장관과 협상을 타결하여 하이포 알페아드리아뱅크 인터내셔널(케른텐주 지역 은행)의 붕괴에서 시작된 양측 정부의 법적 분쟁을 끝냈다. 양해 각서에 따르면 오스트리아는 바이에른주에 12억 3천만 유로를 지불하며, 모든 관련 소송은 취하되었다. \n\n2012년 죄더는 제호퍼 당시 주총리와 함께 연방헌법법원에 소송을 제기하여 바이에른처럼 부유한 주가 전국의 부실경제 구제 차원에서 재정이전을 하도록 하는 독일 시스템 점검을 요구했다. 죄더의 제안에 따라 바이에른주 정부는 독일 최초로 폭스바겐을 상대로 배출가스 시험 사기 사건 관련 소송을 제기해 손해배상을 요구한 주가 되었다. 이 시기 죄더는 해당 스캔들로 인해 70만 유로에 달하는 공무원 연금기금 손실을 입었다고 밝혔다. \n\n2017년 총선 결과 기사련이 참패하면서 제호퍼는 대표직 사퇴 압력을 받게 되었고, 이에 그는 당대표직에서 물러나지는 않는 대신 바이에른주 총리직은 죄더에게 인계하겠다고 밝혔다.
Top-2 passage with score 172.088379
일본은 한국과 마찬가지로 형사소송법상의 기소편의주의에 의해 사건에 대한 불기소처분을 검찰관(검사)의 재량에 따라 할 수 있다. 하지만 고소한 사람이 이에 불복하는 경우 민간인으로 이루어진 검찰심사회에 불기소 처분이 타당한지에 대한 심사를 요청할 수 있으며, 이는 민의를 이용한 

In [13]:
topK_list = [1,10,20,50]
result = dense_retriever.topk_experiment(topK_list, org_dataset['train'])
print(result)

100%|██████████| 988/988 [01:23<00:00, 11.87it/s]


[query exhaustive search] done in 144.721 s


 25%|██▌       | 1/4 [02:25<07:15, 145.19s/it]

100%|██████████| 988/988 [01:16<00:00, 12.83it/s]


[query exhaustive search] done in 140.164 s


...

[query exhaustive search] done in 112.553 s


100%|██████████| 988/988 [01:14<00:00, 13.27it/s]


[query exhaustive search] done in 138.065 s


100%|██████████| 4/4 [08:58<00:00, 134.70s/it]


{'train_topk_1': 0.4091599190283401, 'train_topk_10': 0.6700404858299596, 'train_topk_20': 0.7332995951417004, 'train_topk_50': 0.8170546558704453}


In [14]:
topK_list = [1,10,20,50]
result = dense_retriever.topk_experiment(topK_list, org_dataset['validation'])
print(result)

100%|██████████| 60/60 [00:03<00:00, 15.13it/s]


[query exhaustive search] done in 6.737 s


 25%|██▌       | 1/4 [00:06<00:20,  6.79s/it]

100%|██████████| 60/60 [00:04<00:00, 14.92it/s]


[query exhaustive search] done in 8.338 s


...

 75%|███████▌  | 3/4 [00:22<00:07,  7.17s/it]

100%|██████████| 60/60 [00:03<00:00, 16.21it/s]


[query exhaustive search] done in 8.133 s


100%|██████████| 4/4 [00:30<00:00,  7.61s/it]


{'train_topk_1': 0.26666666666666666, 'train_topk_10': 0.525, 'train_topk_20': 0.6, 'train_topk_50': 0.6791666666666667}


In [16]:
topK_list = [200]
result = dense_retriever.topk_experiment(topK_list, org_dataset['train'])
print(result)

100%|██████████| 988/988 [01:10<00:00, 14.00it/s]


[query exhaustive search] done in 130.738 s


100%|██████████| 1/1 [02:12<00:00, 132.04s/it]


{'train_topk_200': 0.9147267206477733}
